In [1]:
import pathlib

import numpy as np
import torch
import torchvision
from torch_scae_experiments.newds.train import train
from torch_scae_experiments.newds.hparams import model_params
from pytorch_lightning import seed_everything

seed_everything(42)

42

In [2]:
from pprint import pprint
# Customize model parameters

# model_params.update(dict(
#     n_part_caps=36,
#     n_obj_caps=10,
#     pcae_template_generator_params=dict(
#         template_size=(11, 11),
#         template_nonlin='sigmoid',
#         colorize_templates=False,
#     ),
#     scae_params=dict(
#         posterior_sparsity_loss_type='entropy',
#         posterior_within_example_sparsity_weight=1,
#         posterior_between_example_sparsity_weight=1,
#     )
# ))
pprint(model_params)


{'image_shape': (1, 40, 40),
 'n_classes': 10,
 'n_obj_caps': 32,
 'n_part_caps': 40,
 'ocae_decoder_capsule_params': {'allow_deformations': True,
                                 'caps_dropout_rate': 0.0,
                                 'dim_caps': 32,
                                 'hidden_sizes': (128,),
                                 'learn_vote_scale': True,
                                 'noise_scale': 4.0,
                                 'noise_type': 'uniform',
                                 'similarity_transform': False},
 'ocae_encoder_set_transformer_params': {'dim_hidden': 16,
                                         'dim_out': 256,
                                         'layer_norm': True,
                                         'n_heads': 1,
                                         'n_layers': 3},
 'pcae_cnn_encoder_params': {'activate_final': True,
                             'kernel_sizes': [3, 3, 3, 3],
                             'out_channels': [128, 1

In [3]:
# you can tune the training params here :
training_params = dict(
    data_dir=str(pathlib.Path.home() / 'torch-datasets'),
    gpus=1,
    batch_size=128,
    num_workers=2,
    max_epochs=1,
    learning_rate=3e-5,
    optimizer_type='RMSprop',
    use_lr_scheduler=True,
    lr_scheduler_decay_rate=0.997
)


In [4]:
# first phase of training
torch.cuda.reset_max_memory_cached()
torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_accumulated_memory_stats()

train(model_params, **training_params)

C:\Users\t-nakara\Miniconda3\envs\cs236781-proj\lib\site-packages\torch\cuda\memory.py:257: FutureWarning: torch.cuda.reset_max_memory_cached now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(
C:\Users\t-nakara\Miniconda3\envs\cs236781-proj\lib\site-packages\torch\cuda\memory.py:231: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(
GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
0it [00:00, ?it/s]

170500096it [02:48, 1010407.45it/s]                                                                                                                                                                                                                                                      


Extracting C:\Users\t-nakara\torch-datasets\cifar-10-python.tar.gz to C:\Users\t-nakara\torch-datasets
Files already downloaded and verified



    | Name                                              | Type                      | Params
--------------------------------------------------------------------------------------------
0   | scae                                              | SCAE                      | 3 M   
1   | scae.part_encoder                                 | CapsuleImageEncoder       | 571 K 
2   | scae.part_encoder.encoder                         | CNNEncoder                | 444 K 
3   | scae.part_encoder.encoder.network                 | Sequential                | 444 K 
4   | scae.part_encoder.encoder.network.0               | Conv2d                    | 1 K   
5   | scae.part_encoder.encoder.network.1               | ReLU                      | 0     
6   | scae.part_encoder.encoder.network.2               | Conv2d                    | 147 K 
7   | scae.part_encoder.encoder.network.3               | ReLU                      | 0     
8   | scae.part_encoder.encoder.network.4               | Conv2d     

Validation sanity check: 0it [00:00, ?it/s]

RuntimeError: Given groups=1, weight of size [128, 1, 3, 3], expected input[128, 3, 32, 32] to have 1 channels, but got 3 channels instead

In [ ]:
# resume training
training_params.update(
    max_epochs=200,
    resume_from_checkpoint= '/content/lightning_logs/version_0/checkpoints/epoch=99.ckpt')
)
train(model_params, **training_params)